In [1]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import ta  # For technical analysis indicators

def get_data(tickers, start_date, end_date):
    """Downloads historical price data for multiple tickers using yfinance."""
    data = {}
    for ticker in tickers:
        df = yf.download(ticker, start=start_date, end=end_date)
        if not df.empty:
            data[ticker] = df
        else:
            print(f"Could not retrieve data for {ticker} between {start_date} and {end_date}.")
    return data

def calculate_rsi(data, window=14):
    """Calculates the Relative Strength Index (RSI) for each ticker."""
    for ticker, df in data.items():
        if 'Close' in df.columns:
            df['RSI'] = ta.momentum.rsi(df['Close'].squeeze(), window=window)
        else:
            print(f"Warning: 'Close' column not found in data for {ticker}. Cannot calculate RSI.")
            df['RSI'] = None  # Or handle the missing data as appropriate
    return data

def generate_signals(data):
    """Generates buy and sell signals based on RSI levels."""
    for ticker, df in data.items():
        df['Signal'] = 0
        buy_condition = df['RSI'].shift(1) > 30 and df['RSI'] <= 30
        sell_condition = df['RSI'].shift(1) < 70 and df['RSI'] >= 70
        df['Signal'][buy_condition] = 1  # Buy
        df['Signal'][sell_condition] = 0  # Sell/Exit Long
        df['Position'] = df['Signal'].ffill()
    return data

def calculate_returns(data):
    """Calculates asset and strategy returns for each ticker."""
    for ticker, df in data.items():
        df['Asset_Return'] = df['Close'].pct_change()
        df['Strategy_Return'] = df['Position'].shift(1) * df['Asset_Return']
        df['Cumulative_Asset_Return'] = (1 + df['Asset_Return']).cumprod() - 1
        df['Cumulative_Strategy_Return'] = (1 + df['Strategy_Return']).cumprod() - 1
    return data

def evaluate_performance(data):
    """Prints basic performance metrics for each ticker."""
    print("Performance Metrics:")
    for ticker, df in data.items():
        if not df.empty and 'Cumulative_Strategy_Return' in df.columns and 'Cumulative_Asset_Return' in df.columns:
            total_strategy_return = df['Cumulative_Strategy_Return'].iloc[-1]
            total_asset_return = df['Cumulative_Asset_Return'].iloc[-1]
            print(f"- {ticker}:")
            print(f"  Total Strategy Return: {total_strategy_return:.2%}")
            print(f"  Total Buy and Hold Return: {total_asset_return:.2%}")
        else:
            print(f"- {ticker}: Could not calculate performance metrics due to missing data.")

def visualize_results(data, short_window, long_window, tickers): # Adjusted parameters
    # ... (you would likely modify this to plot RSI instead of SMAs)
    for ticker, df in data.items():
        if not df.empty and 'RSI' in df.columns and 'Close' in df.columns and 'Signal' in df.columns:
            plt.figure(figsize=(12, 6))
            plt.plot(df.index, df['Close'], label=f'{ticker} Close Price')
            plt.plot(df.index, df['RSI'], label=f'{ticker} RSI (14)')
            plt.axhline(30, color='r', linestyle='--', alpha=0.7)
            plt.axhline(70, color='g', linestyle='--', alpha=0.7)
            buy_signals = df[df['Signal'] == 1]
            sell_signals = df[df['Signal'] == 0].iloc[1:]
            plt.scatter(buy_signals.index, df['Close'][buy_signals.index], marker='^', color='g', label='Buy Signal')
            plt.scatter(sell_signals.index, df['Close'][sell_signals.index], marker='v', color='r', label='Sell Signal')
            plt.title(f'RSI Based Strategy for {ticker}')
            plt.xlabel('Date')
            plt.ylabel('Price / RSI')
            plt.legend()
            plt.grid(True)
            plt.show()
        else:
            print(f"Could not visualize RSI strategy for {ticker} due to missing data.")

if __name__ == "__main__":
    tickers = ["AAPL"]
    start_date = "2020-01-01"
    end_date = "2024-12-31"

    all_data = get_data(tickers, start_date, end_date)

    if all_data:
        processed_data = calculate_rsi(all_data)
        processed_data = generate_signals(processed_data)
        processed_data = calculate_returns(processed_data)
        evaluate_performance(processed_data)
        visualize_results(processed_data, None, None, tickers)
    else:
        print("No data retrieved for any of the specified tickers.")


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Could not retrieve data for AAPL between 2020-01-01 and 2024-12-31.
No data retrieved for any of the specified tickers.
